In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

from pydantic import BaseModel
from typing import Optional, Union, Any
from datetime import datetime
from pprint import pprint

In [3]:
class Scene(BaseModel):
    name: str
    location_tags: list[str]
    time_tags: list[str]
    other_tags: list[str]

class Scenes(BaseModel):
    scenes: list[Scene]

scenes_list = ["출퇴근길", "근무", "휴식", "공부", "게임", "유튜브 시청", "애완동물 돌보기"]

In [4]:
class User(BaseModel):
    name: str
    location: str
    birthdate: datetime
    occupation: str
    personality: list[str]
    scenes: list[Scene] = []
    prompt: str
    positives: list[str]
    negatives: list[str]
    
user_context = {
    'name': '윤형석',
    'location': '서울',
    'birthdate': datetime.strptime('1990-03-28', '%Y-%m-%d'),
    'occupation': '예비창업자',
    'personality': ['Introverted', 'Intuitive', 'Thinking', 'Perceiving'],
    'prompt': 'Your prompt here',
    'positives': ['지적 호기심', '창의력', '직관성'],
    'negatives': ['계획성 부족', '조직력 부족', '무기력함']
}

user = User(**user_context)

user_bio = user.model_dump_json()

In [5]:
import json

pydantic_parser = PydanticOutputParser(pydantic_object=Scenes)
scene_parser = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5)
scene_prompt = ChatPromptTemplate.from_template("""
다음은 사용자의 하루를 구성하는 장면들입니다.
사용자의 정보와 이 장면들을 바탕으로, 각각의 장면에 어울리는 시간, 공간, 기타 태그를 3~5개씩 필요에 따라 작성하고, 아래 지침에 따라 json 포맷으로 반환하세요.

태그를 붙이는 목적은 할 일을 관리하기 위한 데이터베이스에 사용하기 위해서입니다.
각각의 할 일은 사용자의 하루를 나타내는 여러 장면을 담은 태그와 함께 저장되고, 사용자가 처한 맥락과 상황을 표현하는 태그에 맞춰 할 일을 추천합니다.

시간 태그에는 휴일 여부, 요일, 하루 중의 시간대 등의 정보를 포함하세요.
공간 태그에는 사용자의 위치, 활동하는 장소 등의 정보를 포함하세요.
기타 태그에는 시간과 공간 태그에 포함되지 않지만 할 일의 맥락과 상황을 검색하기에 좋은 정보를 포함하세요.

사용자 정보: {bio}
장면: {scenes}
지침: {format_instruction}
""").partial(format_instruction=pydantic_parser.get_format_instructions())

scenes_chain = scene_prompt | scene_parser | pydantic_parser
response = scenes_chain.invoke({"bio": user_bio, "scenes": scenes_list})

for scene in response.scenes:
    user.scenes.append(scene)
    print(scene.model_dump_json())

{"name":"출퇴근길","location_tags":["서울","지하철","버스","도로"],"time_tags":["주중","오전","오후","출근 시간","퇴근 시간"],"other_tags":["대중교통","혼잡함","소음"]}
{"name":"근무","location_tags":["사무실","서울","작업 공간"],"time_tags":["주중","오전","오후","업무 시간"],"other_tags":["팀워크","프로젝트","회의"]}
{"name":"휴식","location_tags":["집","서울","카페"],"time_tags":["주중","오후","저녁","짧은 휴식"],"other_tags":["자기 관리","재충전","편안함"]}
{"name":"공부","location_tags":["집","도서관","카페"],"time_tags":["주중","오후","저녁","주말"],"other_tags":["자기 개발","지식 습득","집중"]}
{"name":"게임","location_tags":["집","PC방"],"time_tags":["주말","오후","저녁","여가 시간"],"other_tags":["오락","스트레스 해소","사회적 상호작용"]}
{"name":"유튜브 시청","location_tags":["집"],"time_tags":["주중","저녁","여가 시간"],"other_tags":["취미","정보 습득","오락"]}
{"name":"애완동물 돌보기","location_tags":["집"],"time_tags":["주중","오전","오후","저녁"],"other_tags":["책임감","애정","스트레스 해소"]}


In [6]:
def collect_tags(user):
    location_tags = []
    time_tags = []
    other_tags = []

    for scene in user.scenes:
        location_tags += scene.location_tags
        time_tags += scene.time_tags
        other_tags += scene.other_tags

    location_tags = list(set(location_tags))
    time_tags = list(set(time_tags))
    other_tags = list(set(other_tags))

    return location_tags, time_tags, other_tags

collect_tags(user)

(['지하철', '도로', '버스', '사무실', '서울', '작업 공간', '카페', '집', '도서관', 'PC방'],
 ['저녁', '출근 시간', '업무 시간', '주중', '오전', '퇴근 시간', '주말', '여가 시간', '짧은 휴식', '오후'],
 ['프로젝트',
  '지식 습득',
  '재충전',
  '사회적 상호작용',
  '혼잡함',
  '집중',
  '취미',
  '책임감',
  '자기 개발',
  '스트레스 해소',
  '회의',
  '편안함',
  '정보 습득',
  '팀워크',
  '오락',
  '소음',
  '대중교통',
  '자기 관리',
  '애정'])

In [7]:
from langchain.docstore.document import Document

def store_tags(tag_list, tag_category):
    documents = []
    
    for i, tag in enumerate(tag_list):
        document = Document(page_content=tag, metadata={'source': f'{tag_category}_{i+1}'})
        documents.append(document)

    vectorstore = Chroma.from_documents(
        documents = documents,
        embedding = OpenAIEmbeddings(model = 'text-embedding-3-large'),
        collection_name = f'{tag_category}_tags',
        persist_directory = 'data'
    )
    
    return vectorstore

location_tag_vectors = store_tags(collect_tags(user)[0], 'location')
time_tag_vectors = store_tags(collect_tags(user)[1], 'time')
other_tag_vectors = store_tags(collect_tags(user)[2], 'other')

In [8]:
def print_answer_to_query(query, vectorstore, show_all = False):
    try:
        if vectorstore._collection.count() == 0:
            print("벡터 저장소가 비어 있습니다.")
            return
            
        print(f'벡터 저장소에 저장된 태그의 수: {vectorstore._collection.count()}')
        results = vectorstore.similarity_search_with_score(query)
        
        if results:
            print(f"검색 결과의 길이: {len(results)}")
            print(f"가장 유사한 결과: {results[0][0].page_content} (벡터 거리: {results[0][1]:.4f})\n")

            if show_all:
                for i, (doc, score) in enumerate(results):
                    print(f"결과 {i + 1}: {doc.page_content}")
                    print(f"벡터 거리: {score:.4f}\n")
        else:
            print("검색 결과가 없습니다.")
            
    except Exception as e:
        print(f"오류 발생: {e}")

print_answer_to_query("회사에서 잔업하기", other_tag_vectors, show_all=True)

벡터 저장소에 저장된 태그의 수: 19
검색 결과의 길이: 4
가장 유사한 결과: 자기 관리 (벡터 거리: 1.3196)

결과 1: 자기 관리
벡터 거리: 1.3196

결과 2: 팀워크
벡터 거리: 1.3275

결과 3: 혼잡함
벡터 거리: 1.3459

결과 4: 정보 습득
벡터 거리: 1.3483



In [9]:
import json

user_bio = user.model_dump_json()
user_dict = json.loads(user_bio)
user_dict['scenes'][5]

{'name': '유튜브 시청',
 'location_tags': ['집'],
 'time_tags': ['주중', '저녁', '여가 시간'],
 'other_tags': ['취미', '정보 습득', '오락']}

In [10]:
# LLM 초기화
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=None)

In [11]:
from langchain.schema import BaseOutputParser

class CustomListOutputParser(BaseOutputParser):
    def parse(self, text: str) -> list[str]:
        responses = text.split("---")
        items = [response.strip() for response in responses]
        return items

    def get_format_instructions(self) -> str:
        return '서로 다른 답변은 "---"로 구분하세요.'

# 파서 초기화
response_parser = CustomListOutputParser()

In [12]:
prompt_template = ChatPromptTemplate.from_template("""
다음은 사용자 정보입니다. 이 정보를 바탕으로, 사용자의 성격과 하루 일과, 주요 관심사를를 상상해서 1문단으로 작성하세요.
이를 작성하는 이유는 사용자의 할 일을 사용자의 생활패턴과 맥락에 맞게 구체화하여 추천하기 위해서입니다.
사용자에 대한 이해가 깊어질수록 사용자에게 더 유용한 할 일을 추천할 수 있습니다.
사용자의 긍정적인 면과 부정적인 면을 모두 포함할 수 있도록 작성하세요.

작성된 내용 중 사용자가 적합한 것을 선택할 수 있도록, 서로 다른 내용의 답변을 3~5개 생성하세요.
각각의 답변은 사용자 정보의 다른 부분에 집중하며, 서로 비슷하지 않은 내용이어야 합니다.
예를 들어 한 답변이 "대중교통"이라는 키워드에 집중한다면, 다른 답변은 "도서관" 등 다른 맥락에 집중할 수 있습니다.
만약 비슷한 답변이 생성된다면 생략하세요.

{format_instruction}

사용자 정보: {bio}

답변:
""").partial(format_instruction=response_parser.get_format_instructions())

chain = prompt_template | llm | response_parser
responses = chain.invoke({"bio": user_bio})

In [13]:
for response in responses:
    print(response)
    print("---")

윤형석은 서울에 거주하는 예비창업자로, 내향적이고 직관적인 성격을 지니고 있습니다. 그는 지적 호기심이 강해 새로운 아이디어를 탐구하는 것을 즐기지만, 때때로 계획성과 조직력이 부족해 목표 달성에 어려움을 겪기도 합니다. 출퇴근길에는 대중교통을 이용하며 혼잡한 환경 속에서도 자신의 생각에 몰두하는 편입니다. 업무 시간에는 팀워크를 통해 프로젝트를 진행하며, 저녁 시간에는 집에서 유튜브를 시청하거나 애완동물을 돌보며 재충전하는 시간을 가집니다. 주말에는 게임을 통해 스트레스를 해소하고 사회적 상호작용을 즐깁니다.
---
윤형석은 서울의 혼잡한 대중교통을 이용하며 출퇴근하는 예비창업자입니다. 그는 내향적인 성격 덕분에 혼잡한 지하철이나 버스에서도 자신의 생각에 집중할 수 있는 능력을 가지고 있지만, 때때로 계획성 부족으로 인해 출퇴근 시간에 여유를 두지 못하는 경우가 있습니다. 이러한 상황에서 그는 대중교통을 이용하며 책이나 팟캐스트를 통해 지식을 쌓는 것을 선호합니다.
---
윤형석은 주중 저녁 시간에 집에서 유튜브를 시청하며 여가를 즐기는 예비창업자입니다. 그는 지적 호기심이 강해 다양한 주제의 영상을 탐색하는 것을 좋아하지만, 때때로 무기력함을 느껴 계획적으로 시간을 관리하지 못하는 경향이 있습니다. 이러한 여가 시간은 그에게 스트레스를 해소하고 창의력을 자극하는 중요한 순간이 됩니다.
---
윤형석은 주말 오후에 집에서 게임을 즐기며 스트레스를 해소하는 예비창업자입니다. 그는 내향적인 성격 덕분에 혼자서 게임을 하며 편안함을 느끼지만, 때때로 사회적 상호작용이 부족해 외로움을 느끼기도 합니다. 게임을 통해 친구들과 소통하며 즐거운 시간을 보내는 것이 그에게는 중요한 여가 활동입니다.
---
윤형석은 주중 오후에 집에서 공부하며 자기 개발에 힘쓰는 예비창업자입니다. 그는 직관적이고 창의적인 성격 덕분에 새로운 지식을 습득하는 데 큰 흥미를 느끼지만, 계획성 부족으로 인해 공부 시간을 자주 놓치는 경우가 있습니다. 이러한 자기 개발 시간은 그에게 큰 만족감을 

In [14]:
def set_prompt(user: User, responses: list[str], index: int):
    if index < 0 or index >= len(responses):
        raise IndexError("Index out of bounds.")
    user.prompt = responses[index].strip()

set_prompt(user, responses, 3)
user.prompt

'윤형석은 주말 오후에 집에서 게임을 즐기며 스트레스를 해소하는 예비창업자입니다. 그는 내향적인 성격 덕분에 혼자서 게임을 하며 편안함을 느끼지만, 때때로 사회적 상호작용이 부족해 외로움을 느끼기도 합니다. 게임을 통해 친구들과 소통하며 즐거운 시간을 보내는 것이 그에게는 중요한 여가 활동입니다.'

In [ ]:
class Subtask(BaseModel):
    # basic information of the subtask
    name: str
    id: Optional[Union[int, str]]
    index: Optional[int]
    context: Optional[str]
    
    # tags for the subtask
    location_tags: list[str]
    time_tags: list[str]
    other_tags: list[str]
    estimated_minutes: int
    
    # subtasks of the subtask
    has_subtasks: bool = False
    subtasks: Optional[list['Subtask']]
    
    # super of the subtask
    supertask_id: Optional[Union[int, str]]
    supertask_type: Optional[str]
    
    _supertask: Any = None
    
    def get_supertask(self):
        return self._supertask
    
    def set_supertask(self, task:Any, task_type: str):
        self._supertask = task
        self.supertask_id = getattr(task, 'id', None)
        self.supertask_type = task_type
        
    def set_supertask_of_subtask(self):
        if self.has_subtasks:
            for subtask in self.subtasks:
                subtask.set_supertask(self, 'subtask')
                subtask.set_supertask_of_subtask()
    
    # add a subtask of this subtask
    def add_subtask(self, subtask: 'Subtask'):
        if not self.has_subtasks:
            self.subtasks = []
            self.has_subtasks = True
        self.subtasks.append(subtask)
    
    # set the index of the subtasks
    def set_subtasks_index(self):
        for i, subtask in enumerate(self.subtasks):
            subtask.index = (i + 1)
    
    # print the subtask with all its subtasks
    def print_self(self):
        print(f"Subtask_{self.index}: {self.name}")
        print(f"- Context: {self.context}")
        print(f"- Location Tags: {self.location_tags}")
        print(f"- Time Tags: {self.time_tags}")
        print(f"- Other Tags: {self.other_tags}")
        print(f"- Estimated Minutes: {self.estimated_minutes}")
        print(f"- Supertask: {self.supertask_id} ({self.supertask_type})")
        print()
        
        if self.has_subtasks:
            print("Subtasks:")
            for subtask in self.subtasks:
                subtask.print_self()
    
    # get a subtask of this subtask
    def get_subtask(self, index: int):
        if index < 0 or index >= len(self.subtasks):
            raise IndexError("Index out of bounds.")
        return self.subtasks[index]
    
    # get all subtasks of this subtask
    def get_all_subtasks(self):
        subtasks = []
        
        if self.has_subtasks:
            subtasks = self.subtasks.copy()    
            for subtask in self.subtasks:
                subtasks += subtask.get_all_subtasks()
            
        return subtasks
    
    # set the total minutes of the subtask based on its subtasks if it has any
    def set_total_estimated_minutes(self):
        if self.has_subtasks:
            self.estimated_minutes = sum([subtask.estimated_minutes for subtask in self.subtasks])
            
            for subtask in self.subtasks:
                subtask.set_total_estimated_minutes()

    # update a subtask of this subtask            
    def update_subtask(self, index: int, subtask: 'Subtask'):
        if index < 0 or index >= len(self.subtasks):
            raise IndexError("Index out of bounds.")
        self.subtasks[index] = subtask
    
    # remove a subtask of this subtask
    def remove_subtask(self, index: int):
        if index < 0 or index >= len(self.subtasks):
            raise IndexError("Index out of bounds.")
        del self.subtasks[index]
        self.set_subtasks_index()
    
    # clear all subtasks of this subtask recursively
    def clear_subtasks(self):
        for subtask in self.subtasks:
            subtask.clear_subtasks()
        
        self.subtasks.clear()
        self.has_subtasks = False

    # count the number of subtasks of this subtask
    def count_subtasks(self):
        return len(self.subtasks) if self.has_subtasks else 0

class Task(BaseModel):
    name: str
    id: Optional[Union[int, str]]
    context: str

    location_tags: list[str]
    time_tags: list[str]
    other_tags: list[str]
    estimated_minutes: int
    
    subtasks: list[Subtask]
    
    def set_supertask_of_subtask(self):
        for subtask in self.subtasks:
            subtask.set_supertask(self, 'task')
            subtask.set_supertask_of_subtask()
    
    def add_subtask(self, subtask: Subtask):
        self.subtasks.append(subtask)
        
    def set_subtask_index(self):
        for i, subtask in enumerate(self.subtasks):
            subtask.index = (i + 1)
        
    def get_subtask(self, index: int):
        if index < 0 or index >= len(self.subtasks):
            raise IndexError("Index out of bounds.")
        return self.subtasks[index]
    
    def get_all_subtasks(self):
        subtasks = []
        
        for subtask in self.subtasks:
            subtasks.append(subtask)
            subtasks.extend(subtask.get_all_subtasks())
        
        return subtasks
    
    def set_supertask(self):
        for subtask in self.subtasks:
            subtask.supertask = self
            subtask.set_supertask()
    
    def print_self(self):
        print(f"Task: {self.name}")
        print(f"- Context: {self.context}")
        print(f"- Location Tags: {self.location_tags}")
        print(f"- Time Tags: {self.time_tags}")
        print(f"- Other Tags: {self.other_tags}")
        print(f"- Estimated Minutes: {self.estimated_minutes}")
        print()
        
        print("Subtasks:")
        for subtask in self.subtasks:
            subtask.print_self()
    
    def get_total_minutes(self):
        for subtask in self.subtasks:
            subtask.set_total_estimated_minutes()
        
        self.estimated_minutes = sum([subtask.estimated_minutes for subtask in self.subtasks])
    
    def update_subtask(self, index: int, subtask: Subtask):
        if index < 0 or index >= len(self.subtasks):
            raise IndexError("Index out of bounds.")
        self.subtasks[index] = subtask

    def remove_subtask(self, index: int):
        if index < 0 or index >= len(self.subtasks):
            raise IndexError("Index out of bounds.")
        del self.subtasks[index]

    def clear_subtasks(self):
        self.subtasks.clear()

In [16]:
Subtask.model_rebuild()
Task.model_rebuild()
subtask_pydantic_parser = PydanticOutputParser(pydantic_object=Subtask)
tasks_pydantic_parser = PydanticOutputParser(pydantic_object=Task)

In [17]:
task_breakdown = ChatPromptTemplate.from_template("""
다음은 사용자가 입력한 해야 할 일입니다. 이 할 일을 세부적으로 나누어 작성하세요.
각각의 할 일은 최대한 구체적인 행동으로 작성하고, 이를 수행하는 데 필요한 시간과 노력을 고려하여 작성하세요.
할 일을 세부적으로 나누면 사용자가 할 일을 더 쉽게 완료할 수 있습니다.

각각의 할 일은 사용자의 하루를 나타내는 여러 장면을 담은 태그와 함께 저장되고, 사용자가 처한 맥락과 상황을 표현하는 태그에 맞춰 할 일을 추천합니다.
각각의 할 일은 사용자의 위치, 활동하는 장소, 시간대 등을 고려하여 태그를 부여하세요.

Task의 context에서는 사용자 정보와 하루 일과에 기반하여 해당 할 일을 언제 하는 것이 좋을지 설명하세요.
각각의 Subtask에 적절한 태그를 부여하세요.

{format_instruction}

사용자의 인적 정보: {bio}
사용자의 하루 일과: {prompt}
사용자가 입력한 할 일: {task}
""").partial(format_instruction=tasks_pydantic_parser.get_format_instructions())

task_chain = task_breakdown | llm | tasks_pydantic_parser
task_response = task_chain.invoke({"bio": user_bio, "prompt": user.prompt, "task": "회사에서 잔업하기"})

In [18]:
Task = task_response
Task.set_subtask_index()
Task.get_total_minutes()
Task.set_supertask_of_subtask()
Task.print_self()

Task: 회사에서 잔업하기
- Context: 윤형석은 주중 오후에 사무실에서 잔업을 하며 업무를 마무리하는 시간을 가집니다. 이 시간은 업무 집중도가 높고, 팀원들과의 협업이 필요한 경우가 많습니다.
- Location Tags: ['사무실', '서울']
- Time Tags: ['주중', '오후', '업무 시간']
- Other Tags: ['팀워크', '프로젝트', '회의']
- Estimated Minutes: 120

Subtasks:
Subtask_1: 잔업할 업무 목록 정리하기
- Context: 잔업을 시작하기 전에 어떤 업무를 처리할지 목록을 정리합니다.
- Location Tags: ['사무실']
- Time Tags: ['주중', '오후', '업무 시간']
- Other Tags: ['업무 계획', '조직']
- Estimated Minutes: 15
- Supertask: 1 (task)

Subtask_2: 우선순위에 따라 업무 처리하기
- Context: 정리한 업무 목록을 바탕으로 우선순위에 따라 업무를 처리합니다.
- Location Tags: ['사무실']
- Time Tags: ['주중', '오후', '업무 시간']
- Other Tags: ['업무 집중', '효율성']
- Estimated Minutes: 60
- Supertask: 1 (task)

Subtask_3: 팀원과의 소통 및 피드백 받기
- Context: 업무 진행 중 필요한 경우 팀원과 소통하여 피드백을 받습니다.
- Location Tags: ['사무실']
- Time Tags: ['주중', '오후', '업무 시간']
- Other Tags: ['팀워크', '소통']
- Estimated Minutes: 30
- Supertask: 1 (task)

Subtask_4: 업무 마무리 및 정리하기
- Context: 잔업이 끝난 후, 업무를 마무리하고 정리합니다.
- Location Tags: ['사무실']
- Time Tags: ['주중', '오후', '업무

In [19]:
Task.get_subtask(0)

Subtask(name='잔업할 업무 목록 정리하기', id=1, index=1, context='잔업을 시작하기 전에 어떤 업무를 처리할지 목록을 정리합니다.', location_tags=['사무실'], time_tags=['주중', '오후', '업무 시간'], other_tags=['업무 계획', '조직'], estimated_minutes=15, has_subtasks=False, subtasks=None, supertask_id=1, supertask_type='task')

In [20]:
subtask_breakdown = ChatPromptTemplate.from_template("""
다음은 사용자가 입력한 해해야 할 일입니다. 이 할 일을 세부적으로 나누어 작성하세요.
각각의 할 일은 최대한 구체적인 행동으로 작성하고, 이를 수행하는 데 필요한 시간과 노력을 고려하여 작성하세요.
할 일을 세부적으로 나누면 사용자가 할 일을 더 쉽게 완료할 수 있습니다.

각각의 할 일은 사용자의 하루를 나타내는 여러 장면을 담은 태그와 함께 저장되고, 사용자가 처한 맥락과 상황을 표현하는 태그에 맞춰 할 일을 추천합니다.
각각의 할 일은 사용자의 위치, 활동하는 장소, 시간대 등을 고려하여 태그를 부여하세요.

Subask의 context에서는 사용자 정보와 하루 일과에 기반하여 해당 할 일을 언제 하는 것이 좋을지 설명하세요.
각각의 Subtask에 적절한 태그를 부여하세요.

{format_instruction}

사용자의 인적 정보: {bio}
사용자의 하루 일과: {prompt}
사용자가 입력한 할 일: {task}
""").partial(format_instruction=subtask_pydantic_parser.get_format_instructions())

subtask_chain = subtask_breakdown | llm | subtask_pydantic_parser
subtask_response = subtask_chain.invoke({"bio": user_bio, "prompt": user.prompt, "task": Task.get_subtask(0)})

In [21]:
subtask_response.print_self()

Subtask_1: 잔업할 업무 목록 정리하기
- Context: 잔업을 시작하기 전에 어떤 업무를 처리할지 목록을 정리합니다.
- Location Tags: ['사무실']
- Time Tags: ['주중', '오후', '업무 시간']
- Other Tags: ['업무 계획', '조직']
- Estimated Minutes: 15
- Supertask: 1 (task)

Subtasks:
Subtask_1: 현재 진행 중인 프로젝트 확인하기
- Context: 현재 진행 중인 프로젝트의 상태를 확인하여 잔업 목록에 포함할 업무를 결정합니다.
- Location Tags: ['사무실']
- Time Tags: ['주중', '오후', '업무 시간']
- Other Tags: ['업무 계획', '조직']
- Estimated Minutes: 5
- Supertask: 1 (task)

Subtask_2: 우선순위 정하기
- Context: 확인한 프로젝트를 바탕으로 우선순위를 정하여 잔업 목록을 작성합니다.
- Location Tags: ['사무실']
- Time Tags: ['주중', '오후', '업무 시간']
- Other Tags: ['업무 계획', '조직']
- Estimated Minutes: 5
- Supertask: 1 (task)

Subtask_3: 업무 목록 작성하기
- Context: 우선순위에 따라 잔업 목록을 작성합니다.
- Location Tags: ['사무실']
- Time Tags: ['주중', '오후', '업무 시간']
- Other Tags: ['업무 계획', '조직']
- Estimated Minutes: 5
- Supertask: 1 (task)



In [22]:
subtask_response.set_total_estimated_minutes()
subtask_response.estimated_minutes

15

In [23]:
Task.update_subtask(0, subtask_response)
Task.get_total_minutes()
Task.set_supertask_of_subtask()
Task.print_self()

Task: 회사에서 잔업하기
- Context: 윤형석은 주중 오후에 사무실에서 잔업을 하며 업무를 마무리하는 시간을 가집니다. 이 시간은 업무 집중도가 높고, 팀원들과의 협업이 필요한 경우가 많습니다.
- Location Tags: ['사무실', '서울']
- Time Tags: ['주중', '오후', '업무 시간']
- Other Tags: ['팀워크', '프로젝트', '회의']
- Estimated Minutes: 120

Subtasks:
Subtask_1: 잔업할 업무 목록 정리하기
- Context: 잔업을 시작하기 전에 어떤 업무를 처리할지 목록을 정리합니다.
- Location Tags: ['사무실']
- Time Tags: ['주중', '오후', '업무 시간']
- Other Tags: ['업무 계획', '조직']
- Estimated Minutes: 15
- Supertask: 1 (task)

Subtasks:
Subtask_1: 현재 진행 중인 프로젝트 확인하기
- Context: 현재 진행 중인 프로젝트의 상태를 확인하여 잔업 목록에 포함할 업무를 결정합니다.
- Location Tags: ['사무실']
- Time Tags: ['주중', '오후', '업무 시간']
- Other Tags: ['업무 계획', '조직']
- Estimated Minutes: 5
- Supertask: 1 (subtask)

Subtask_2: 우선순위 정하기
- Context: 확인한 프로젝트를 바탕으로 우선순위를 정하여 잔업 목록을 작성합니다.
- Location Tags: ['사무실']
- Time Tags: ['주중', '오후', '업무 시간']
- Other Tags: ['업무 계획', '조직']
- Estimated Minutes: 5
- Supertask: 1 (subtask)

Subtask_3: 업무 목록 작성하기
- Context: 우선순위에 따라 잔업 목록을 작성합니다.
- Location Tags: ['사무실']
- Time Tags: 